# Tom and Alex Violet Project 2
# April 13, 2025



In [73]:
import matplotlib.pyplot as plt
from CollabFilterOneVectorPerItem import CollabFilterOneVectorPerItem
from train_valid_test_loader import load_train_valid_test_datasets

import autograd
import autograd.numpy as ag_np
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
import sklearn

from surprise import SVD,KNNBasic,KNNWithZScore,KNNWithMeans,SlopeOne,CoClustering
from surprise import Dataset, Reader, accuracy
from surprise.model_selection import cross_validate
from surprise import model_selection

from surprise import SVD, SVDpp
from surprise import Dataset, Reader
from surprise.model_selection import RandomizedSearchCV

from sklearn.neighrbors import NearestNeighors

import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8')

In [64]:
# ------------------------------------------------------


##LOADING DATA##
reader = Reader(
    line_format='user item rating', sep=',',
    rating_scale=(1, 5), skip_lines=1)

# train_data = Dataset.load_from_file(
#     'data_movie_lens_100k/ratings_all_development_set.csv', reader=reader)
# # print(train_set[:5])
# train_set = train_data.build_full_trainset()
# raw_ratings = train_set.build_testset()


#MAKING MERGE DATA FRAME
df = pd.read_csv('data_movie_lens_100k/ratings_all_development_set.csv')
user_df = pd.read_csv('data_movie_lens_100k/user_info.csv')
movie_df = pd.read_csv('data_movie_lens_100k/movie_info.csv')
merged_df = pd.merge(df, user_df, on='user_id', how='inner')
merged_df = merged_df.drop(columns=['orig_user_id'])
merged_movie_df = pd.merge(merged_df, movie_df, on='item_id', how='inner')



# user_df = pd.read_csv('data_movie_lens_100k/user_info.csv')'
# print(merged_movie_df[:100])
# data = Dataset.load_from_df(df[['user_id', 'item_id', 'rating']], reader)



DATA_PATH = 'data_movie_lens_100k/' # TODO fixme: Path to where dataset csv files live on your system
if not os.path.exists(os.path.join(DATA_PATH, 'select_movies.csv')):
    try:
        DATA_PATH = os.path.join(os.environ.get("HOME", ""),
                    'courses/cs135-25s-staffonly/proj_src/projB/data_movie_lens_100k/')
        assert os.path.exists(os.path.join(DATA_PATH, 'select_movies.csv'))
    except AssertionError:
        print("Please store path to movie_lens_100k dataset in DATA_PATH")
assert os.path.exists(os.path.join(DATA_PATH, 'select_movies.csv'))


df_masked = pd.read_csv("data_movie_lens_100k/ratings_masked_leaderboard_set.csv")

df_masked['user_id'] = df_masked['user_id'].astype(str)
df_masked['item_id'] = df_masked['item_id'].astype(str)
df_masked['rating'] = 0


# Create a list of (uid, iid) pairs
predict_pairs = list(zip(df_masked['user_id'], df_masked['item_id'],df_masked['rating']))


dev_set = Dataset.load_from_file(
   os.path.join(DATA_PATH, 'ratings_all_development_set.csv'), reader=reader)
dev_set_for_fit = dev_set.build_full_trainset()
dev_set_for_predict = dev_set_for_fit.build_testset()
dev_set_for_fit.global_mean

print("Global Mean: ", dev_set_for_fit.global_mean)


Global Mean:  3.529480398257623


In [ ]:
##TRAINING##---------------------------------------

# SVD

# param_grid = {
#     # "n_factors": [20, 50, 100],        # Latent dimensions (embedding size)
#     # "lr_all": [0.002, 0.005, 0.01],    # Learning rate for all parameters
#     # "reg_all": [0.02, 0.05, 0.1],      # Regularization term for all parameters
#     # "n_epochs": [10, 20, 30]
#     "n_factors": [100],        # Latent dimensions (embedding size)
#     "lr_all": [0.01],    # Learning rate for all parameters
#     "reg_all": [0.1],      # Regularization term for all parameters
#     "n_epochs": [30]          # Number of SGD iterations
# }
# model = model_selection.search.RandomizedSearchCV(SVDpp, param_grid, n_iter=1, measures=['mae'], refit=True, n_jobs=-1)
# model.fit(dev_set) 
# print("Lowest MAE: ", model.best_score['mae'])
# print("Best params:", model.best_params['mae'])
# best_model = model.best_estimator['mae']
# best_model.fit(dev_set_for_fit)

##TRAIN AND FIT
# best_model = SVDpp(n_factors=100, lr_all=0.01, reg_all=0.1, n_epochs=30)
# best_model.fit(dev_set_for_fit)


In [66]:
# KNN 
param_grid = {
    'k': [30,40,50,60,70],
    'sim_options': {
        'name': ['cosine', 'pearson', 'pearson_baseline'],
        'user_based': [True, False]  # True = user-user, False = item-item
    },
    'min_k': [1, 3, 5,10,50],
    'verbose': [False]
}
model = model_selection.search.RandomizedSearchCV(KNNWithMeans, param_grid, n_iter=50, measures=['mae'], refit=True, n_jobs=-1)
model.fit(dev_set) 
best_model = model.best_estimator['mae']
best_model.fit(dev_set_for_fit)

predictions = [best_model.predict(uid, iid) for (uid, iid, _) in predict_pairs]
for i in range(25):
    print(predictions[i][2],predictions[i][3])

/Users/tomzhou/micromamba/envs/cs135_25s_env/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


None 3.5198781131099106
None 4.29031920785755
None 3.806859800586653
None 3.672657170089098
None 3.942898410883251
None 4.329172982045815
None 2.1810152072016886
None 3.8409144167772253
None 3.7887285572112375
None 3.5911277618464172
None 3.7855537153118766
None 3.8099355532506207
None 3.336980700997417
None 4.847584586769668
None 3.380682607008766
None 3.0221416573124995
None 2.729279484504305
None 2.88329418277615
None 3.847999268398846
None 3.5942914889431496
None 4.3083333653574165
None 3.3273125939081165
None 3.003300585003246
None 2.6928257644570266
None 2.4807780523026812


In [67]:
##KNNBasic
print(model.best_score)
print(model.best_params)

# print(model.cv_results)


{'mae': 0.7261871781900993}
{'mae': {'k': 30, 'sim_options': {'name': 'pearson_baseline', 'user_based': False}, 'min_k': 1, 'verbose': False}}


In [70]:
# SlopeOne
param_grid = {
    # 'k': [30,40,50,60,70],
    # 'sim_options': {
    #     'name': ['cosine', 'pearson', 'pearson_baseline'],
    #     'user_based': [True, False]  # True = user-user, False = item-item
    # },
    # 'min_k': [1, 3, 5,10,50],
    # 'verbose': [False]
}
model = model_selection.search.RandomizedSearchCV(SlopeOne, param_grid, n_iter=1, measures=['mae'], refit=True, n_jobs=-1)
model.fit(dev_set) 
best_model = model.best_estimator['mae']
best_model.fit(dev_set_for_fit)

predictions = [best_model.predict(uid, iid) for (uid, iid, _) in predict_pairs]
for i in range(25):
    print(predictions[i][2],predictions[i][3])

None 4.155567712121761
None 3.9593949068610064
None 3.4873950558271436
None 3.4989646933203002
None 4.103658957595336
None 4.6563894142931215
None 2.076563929967369
None 3.7939755997580074
None 4.318627531285854
None 3.3033558372760172
None 3.329969042231342
None 3.863420532333943
None 3.6117906928404495
None 4.657285535380868
None 3.267080817685785
None 2.8693063370680068
None 2.3377761942607522
None 2.9527377580572245
None 3.668507171756517
None 3.6466174578502346
None 4.433733655758613
None 3.880831658697207
None 2.7411188217195592
None 3.384586968704414
None 2.3213296198296103


In [72]:
##KNNWITHZSCORE
print(model.best_score)

{'mae': 0.7445032514566294}


In [76]:
# CoClustering
param_grid = {
    'n_cltr_u': [1,3,5,10,20],
    'n_cltr_i': [1,3,5,10,20],
    'n_epochs': [10,20,50,100,500],

}
model = model_selection.search.RandomizedSearchCV(CoClustering, param_grid, n_iter=40, measures=['mae'], refit=True, n_jobs=-1)
model.fit(dev_set) 
best_model = model.best_estimator['mae']
best_model.fit(dev_set_for_fit)

predictions = [best_model.predict(uid, iid) for (uid, iid, _) in predict_pairs]
for i in range(25):
    print(predictions[i][2],predictions[i][3])

None 3.7698338571965544
None 4.281317591217826
None 3.6332487936492712
None 3.686945226948176
None 3.999051374208355
None 4.207190698755753
None 1.8249087048347672
None 3.885546056033921
None 4.4091903363964775
None 3.5130471892335415
None 3.535182622567524
None 3.699789167900948
None 3.3523048042668617
None 4.858192424094522
None 3.5293492710014527
None 2.771079287870912
None 2.56969397777473
None 2.7895301520230613
None 3.6510267875395574
None 3.666729955114779
None 4.736645652336845
None 3.742358965126709
None 2.592809410363746
None 3.0832562216431936
None 2.6731205685805084


In [77]:
##KNNWITHMEAN
print(model.best_score)

{'mae': 0.7494713730975355}


In [ ]:
ratings_only = [pred[3] for pred in predictions]  # extract just the ratings
ratings_only = np.array(ratings_only, dtype=float)
np.savetxt("predicted_ratings_leaderboard.txt", ratings_only, fmt="%.6f")
